In [ ]:
import pandas as pd
df = pd.read_json('https://query.data.world/s/ycff646cmqtoejgn6vdfe5kfhb6tz7')
df_old= df
df=df.drop(['url','crawled_at'], axis=1)
df=df.drop(['address'], axis=1)
df=df.drop(['experience'], axis=1)
df=df.drop(['domain','raw_description','posted_at','longitude','postalCode','latitude','_id'], axis=1)
class LabelEncoderExt(object):
    def __init__(self):
        """.
        It differs from LabelEncoder by handling new classes and providing a value for it [Unknown]
        Unknown will be added in fit and transform will take care of new item. It gives unknown class id
        """
        self.label_encoder = LabelEncoder()
        # self.classes_ = self.label_encoder.classes_

    def fit(self, data_list):
        """
        This will fit the encoder for all the unique values and introduce unknown value
        :param data_list: A list of string
        :return: self
        """
        self.label_encoder = self.label_encoder.fit(list(data_list) + ['Unknown'])
        self.classes_ = self.label_encoder.classes_

        return self

    def transform(self, data_list):
        """
        This will transform the data_list to id list where the new values get assigned to Unknown class
        :param data_list:
        :return:
        """
        new_data_list = list(data_list)
        for unique_item in np.unique(data_list):
            if unique_item not in self.label_encoder.classes_:
                new_data_list = ['Unknown' if x==unique_item else x for x in new_data_list]

        return self.label_encoder.transform(new_data_list)
from sklearn.preprocessing import LabelEncoder
import numpy as np


label_encoder_region = LabelEncoderExt()

label_encoder_region.fit(df['region'])
df['region']=label_encoder_region.transform(df['region'])


label_encoder_locality= LabelEncoderExt()

label_encoder_locality.fit(df['locality'])
df['locality']=label_encoder_locality.transform(df['locality'])


label_encoder_et= LabelEncoderExt()

label_encoder_et.fit(df['employment_type'])
df['employment_type']=label_encoder_et.transform(df['employment_type'])


label_encoder_c= LabelEncoderExt()

label_encoder_c.fit(df['country'])
df['country']=label_encoder_c.transform(df['country'])


label_encoder_cp= LabelEncoderExt()

label_encoder_cp.fit(df['company'])
df['company']=label_encoder_cp.transform(df['company'])

label_encoder_education= LabelEncoderExt()

label_encoder_education.fit(df['education'])
df['education']=label_encoder_education.transform(df['education'])

import random
def get_exp():
    return random.randint(0, 11)
df['experience'] = df.apply(lambda x: get_exp(), axis=1)

df=df.drop(['salary'], axis=1)
from sklearn.cluster import AgglomerativeClustering
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(
   
)


def get_embeddings(x):
    
    return vectorizer.fit_transform(x)
e=[]
for i in df['skills']:
  e.append(i)

d=[]
for i in df['description']:
  d.append(i)
t=[]
for i in df['title']:
  t.append(i)
X_tfidf_e = vectorizer.fit_transform(e)
X_tfidf_d = vectorizer.fit_transform(d)

X_tfidf_t = vectorizer.fit_transform(t)
from sklearn.cluster import KMeans


kmeans_e = KMeans(
        n_clusters=800,
        max_iter=10,
        n_init=1,
    ).fit(X_tfidf_e)
    
kmeans_t = KMeans(
        n_clusters=500,
        max_iter=10,
        n_init=1,
    ).fit(X_tfidf_t)
kmeans_d = KMeans(
        n_clusters=500,
        max_iter=10,
        n_init=1,
    ).fit(X_tfidf_t)
    
import numpy
labels_e = kmeans_e.labels_
labels_d = kmeans_d.labels_
labels_t = kmeans_t.labels_
df['skills_embed']=labels_e
df['desc_embed']=labels_d
df['title_embed']=labels_t
df=df.drop(['skills','description','title'], axis=1)
kmeans = KMeans(n_clusters=95)           
kmeans.fit(df)         
labels = kmeans.labels_
df_mk_cluster = pd.concat([df, pd.DataFrame({'cluster': labels})], axis = 1) 
df_mk_cluster.head()


from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(max_depth=2, random_state=0)
clf.fit(df_mk_cluster[df_mk_cluster.columns[~df_mk_cluster.columns.isin(['cluster'])]].values,df_mk_cluster['cluster'].values)

finaldf=pd.concat([df_mk_cluster,df_old[['skills','title','description']]],axis=1)
